In [5]:
!pip install -q transformers


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install biopython


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

                                              0.0/159.8 MB ? eta -:--:--
                                              0.3/159.8 MB 8.6 MB/s eta 0:00:19
                                              0.5/159.8 MB 6.7 MB/s eta 0:00:24
                                              0.8/159.8 MB 6.6 MB/s eta 0:00:25
                                              1.1/159.8 MB 6.5 MB/s eta 0:00:25
                                              1.4/159.8 MB 7.0 MB/s eta 0:00:23
                                              1.7/159.8 MB 6.9 MB/s eta 0:00:23
                                              2.0/159.8 MB 6.7 MB/s eta 0:00:24
                                              2.3/159.8 MB 6.7 MB/s eta 0:00:24
                                              2.6/159.8 MB 6.7 MB/s eta 0:00:24
                                              2.9/159.8 MB 6.8 MB/s eta 0:00:23
                                              3.2/159.8 MB 6.7 MB/s eta 0:00:24
                                              3


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple

from Bio import Entrez
from Bio.PDB import PDBList
import pandas as pd

from transformers import BertModel, BertTokenizer
import re

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split

import pickle

put the first file here

In [4]:


def download_pdb_files(pdb_ids, output_dir):
  pdbl = PDBList()
  for pdb_id in pdb_ids:
    pdb_id_trimmed = pdb_id[:-1] #the last letter is not pdb id, its the chain we need to take
    pdbl.retrieve_pdb_file(pdb_id_trimmed, pdir=output_dir, file_format='mmCif') #maybe change it to mm.cif?

file_path = 'cullpdb_pc30.0_res0.0-2.0_noBrks_noDsdr_len40-10000_R0.25_Nmr_d2024_06_13_chains2636'
df = pd.read_csv(file_path, delim_whitespace=True)



df = df[df['len'] <= 128]
pdb_ids = df['PDBchain'].tolist()
print(pdb_ids)

download_pdb_files(pdb_ids, 'small proteins')

['1E0ZA', '1RY4A', '1UG7A', '1WHNA', '2AYAA', '2D87A', '2DB5A', '2DKPA', '2EBKA', '2ETTA', '2FM4A', '2I4KA', '2JQ5A', '2K0RA', '2K5TA', '2LR4A', '2LS01', '2LVLA', '2MAHA', '2NCNA', '2RLOA', '2RV8A', '2YTUA', '4AR0A', '5AIWA', '5H7UA', '5N9QA', '8E22X', '1CMOA', '1DDFA', '1EALA', '1PQNA', '1QKLA', '1UEYA', '1W2QA', '1WH4A', '1WJRA', '1X5MA', '1ZU1A', '2CSOA', '2DMKA', '2EE7A', '2L1LB', '2LOEA', '2MF7A', '2MPLA', '2NBOA', '2TMPA', '4UEIA', '5M1UA', '7XRWA', '8OILA', '1RSFA', '1V5SA', '1X5EA', '1ZO0A', '2A4HA', '2CR4A', '2DAVA', '2IF1A', '2JP2A', '2JVNA', '2K2VA', '2L3GA', '2L57A', '2LBCA', '2LLNA', '2M26A', '3MSPA', '7F7NA', '7KNVA', '8SQXA', '1BUQA', '1BW3A', '1FR0A', '1GIOA', '1MH6A', '1PSYA', '1UENA', '1WYJA', '2AI6A', '2DHJA', '2K4VA', '2K4ZA', '2KA5A', '2KFPA', '2KYTA', '2L2CA', '2LDUA', '2LM0A', '2MDKA', '2MV4A', '2N2HB', '2NBGA', '2PRFA', '4A53A', '5KNWA', '5T3YA', '5WOEA', '6GWMA', '6TH8A', '6UF2A', '6XEHA', '7BQCA', '7PRDA', '7QYIA', '1DC7A', '1E8EA', '1EO1A', '1PIRA', '1T3VA', 

C:\Users\sandr\AppData\Local\Temp\ipykernel_37504\864766783.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True)


In [5]:
file_path = 'cullpdb_pc30.0_res0.0-2.0_noBrks_noDsdr_len40-10000_R0.25_Nmr_d2024_06_13_chains2636'
df = pd.read_csv(file_path, delim_whitespace=True)

#pdb_ids = df['PDBchain'].tolist()

df = df[df['len'] <= 128]
pdb_ids = df['PDBchain'].tolist()
print(pdb_ids)

['1E0ZA', '1RY4A', '1UG7A', '1WHNA', '2AYAA', '2D87A', '2DB5A', '2DKPA', '2EBKA', '2ETTA', '2FM4A', '2I4KA', '2JQ5A', '2K0RA', '2K5TA', '2LR4A', '2LS01', '2LVLA', '2MAHA', '2NCNA', '2RLOA', '2RV8A', '2YTUA', '4AR0A', '5AIWA', '5H7UA', '5N9QA', '8E22X', '1CMOA', '1DDFA', '1EALA', '1PQNA', '1QKLA', '1UEYA', '1W2QA', '1WH4A', '1WJRA', '1X5MA', '1ZU1A', '2CSOA', '2DMKA', '2EE7A', '2L1LB', '2LOEA', '2MF7A', '2MPLA', '2NBOA', '2TMPA', '4UEIA', '5M1UA', '7XRWA', '8OILA', '1RSFA', '1V5SA', '1X5EA', '1ZO0A', '2A4HA', '2CR4A', '2DAVA', '2IF1A', '2JP2A', '2JVNA', '2K2VA', '2L3GA', '2L57A', '2LBCA', '2LLNA', '2M26A', '3MSPA', '7F7NA', '7KNVA', '8SQXA', '1BUQA', '1BW3A', '1FR0A', '1GIOA', '1MH6A', '1PSYA', '1UENA', '1WYJA', '2AI6A', '2DHJA', '2K4VA', '2K4ZA', '2KA5A', '2KFPA', '2KYTA', '2L2CA', '2LDUA', '2LM0A', '2MDKA', '2MV4A', '2N2HB', '2NBGA', '2PRFA', '4A53A', '5KNWA', '5T3YA', '5WOEA', '6GWMA', '6TH8A', '6UF2A', '6XEHA', '7BQCA', '7PRDA', '7QYIA', '1DC7A', '1E8EA', '1EO1A', '1PIRA', '1T3VA', 

C:\Users\sandr\AppData\Local\Temp\ipykernel_37504\3284862781.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True)


In [6]:
import shutil


In [7]:
#we need the chain acces information
pdb_id_map = {pdb_id[:-1]: pdb_id for pdb_id in pdb_ids}
print(pdb_id_map)

mapping_file = os.path.join( 'pdb_id_mapping.csv')
#pd.Series(pdb_id_map).to_csv(mapping_file, header=False)
pd.Series({k.lower(): v for k, v in pdb_id_map.items()}).to_csv(mapping_file, header=False)

{'1E0Z': '1E0ZA', '1RY4': '1RY4A', '1UG7': '1UG7A', '1WHN': '1WHNA', '2AYA': '2AYAA', '2D87': '2D87A', '2DB5': '2DB5A', '2DKP': '2DKPA', '2EBK': '2EBKA', '2ETT': '2ETTA', '2FM4': '2FM4A', '2I4K': '2I4KA', '2JQ5': '2JQ5A', '2K0R': '2K0RA', '2K5T': '2K5TA', '2LR4': '2LR4A', '2LS0': '2LS01', '2LVL': '2LVLA', '2MAH': '2MAHA', '2NCN': '2NCNA', '2RLO': '2RLOA', '2RV8': '2RV8A', '2YTU': '2YTUA', '4AR0': '4AR0A', '5AIW': '5AIWA', '5H7U': '5H7UA', '5N9Q': '5N9QA', '8E22': '8E22X', '1CMO': '1CMOA', '1DDF': '1DDFA', '1EAL': '1EALA', '1PQN': '1PQNA', '1QKL': '1QKLA', '1UEY': '1UEYA', '1W2Q': '1W2QA', '1WH4': '1WH4A', '1WJR': '1WJRA', '1X5M': '1X5MA', '1ZU1': '1ZU1A', '2CSO': '2CSOA', '2DMK': '2DMKA', '2EE7': '2EE7A', '2L1L': '2L1LB', '2LOE': '2LOEA', '2MF7': '2MF7A', '2MPL': '2MPLA', '2NBO': '2NBOA', '2TMP': '2TMPA', '4UEI': '4UEIA', '5M1U': '5M1UA', '7XRW': '7XRWA', '8OIL': '8OILA', '1RSF': '1RSFA', '1V5S': '1V5SA', '1X5E': '1X5EA', '1ZO0': '1ZO0A', '2A4H': '2A4HA', '2CR4': '2CR4A', '2DAV': '2DAV

In [39]:
from Bio.PDB import PDBParser
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def is_chain_continuous(chain): # this function is not necessary, just checks if the chains are continuous
    prev_resnum = None
    for residue in chain:
        if prev_resnum is not None and residue.get_id()[1] != prev_resnum + 1:
            return False
        prev_resnum = residue.get_id()[1]
    return True

def extract_input(pdb_file, model, tokenizer):
    #parser = PDBParser()
    #structure = parser.get_structure("protein", pdb_file)
    #model0 = structure[0]

    file_model = file_path.replace(".cif","") # loading file
    pdbl = PDBList()
    cif_parser = MMCIFParser()
    structure = cif_parser.get_structure("protein", pdb_file) # getting structure
    model0 = structure[0]


    mapping_file = os.path.join('pdb_id_mapping.csv') # this is to access the dictionary keys, which are the same as the file name, once the jeys are accessed we can find which chain to take
    pdb_id_map = pd.read_csv(mapping_file, header=None, index_col=0).squeeze().to_dict()

    #extracting the file name as it contains the id
    filename = os.path.basename(pdb_file)
    pdb_id = filename[:4] # the first 4 letters of the file name is the dictionary key,whose value is the chain to be used
    pdb_id = pdb_id
    #print(pdb_id)

    full_pdb_id = pdb_id_map.get(pdb_id) #pdb id with the correct chain


    chain_A = model0[full_pdb_id[-1]]  # and we get chain A , the last letter of the id is the chain


    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    #print(len(sequence))
    #print(''.join(sequence))
    #seq_example =  ' '.join(sequence)
    #return seq_example

#def extract_coordinates(pdb_file):
    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

    chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

    cnt = 1
    phi_angles = {}
    phi_angles_list = []
    psi_angles = {}
    psi_angles_list = []

    for key in d:
      if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
          phi_angles[key] = d[key].angle
          phi_angles_list.append(d[key].angle)
      elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
          psi_angles[key] = d[key].angle
          psi_angles_list.append(d[key].angle)


    structure.internal_to_atom_coordinates(verbose = False)#before it was true, you can change this back if theres any
    io = PDBIO() #this is to write a pdb file again
    io.set_structure(structure)#set structure, the structure you wan tin the pdb file
    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = np.vstack((psi,phi))
    angle_tensor = torch.tensor(angles, dtype=torch.float32)


    #seq = re.sub(r"[UZOB]", "X", seq_example)

    encoded_input = tokenizer(sequence, return_tensors='pt')  #here the tokenizer you use


    #Make predictions
    outputs = model(**encoded_input)

    embeddings = outputs.last_hidden_state

    #print('N:', N,D)
    return sequence, angle_tensor, encoded_input, outputs, embeddings #you may not need to extract sequence info for each file

In [9]:

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False ) # change model and tokenizer to t5
model_embedd = BertModel.from_pretrained("Rostlab/prot_bert")



In [10]:
len(df) # number of proteins we need to extract in PISCES data

1723

You need to upload a huge folder of all the data here. The pdb files folder in directory path. In target path upload the inputs folder

In [46]:
import os
import torch

directory_path = 'small proteins'
target_path = '.'
embeddings_file = os.path.join(target_path, 'embeddings.pt')
angles_file = os.path.join(target_path, 'angles.pt')
sequences_file = os.path.join(target_path, 'sequences.pkl') # you  may not need this

# Load existing tensors if they exist
if os.path.exists(embeddings_file) and os.path.exists(angles_file): # not imp for your case, checking if we already have files from previous uncrashed runtimes
    embeddings = torch.load(embeddings_file)
    angles = torch.load(angles_file)
    with open(sequences_file, 'rb') as f:
        sequences = pickle.load(f)
    #print(embeddings.size(0))
    start_index = embeddings.size(0)

else:
    #notice i have padded embeddings so they are all the same size. i am not sure if that will cause issues
    # as im making everything zero, and later filling it, automatically the unfilled parts are padded with zeros
    embeddings = torch.zeros(len(df), 130, 1024) # embeddings max size is 130, because you have astart and an end token

    angles = torch.zeros(len(df), 2, 130)
    sequences = []
    start_index = 0

# Number of files to process in this run
num_files_to_process = 3 #then 90

# Iterate over the files to process

i = start_index

def pad_sequence(sequence, target_length=129, pad_char="X"): # may not need this as im using a padding in the tokenizer itself
    padding_length = target_length - len(sequence)
    padding = [pad_char] * padding_length
    return sequence + padding

# Function to get last modified time of a file
def get_file_modified_time(file_path):
    return os.path.getmtime(file_path)

# Get list of files sorted by last modified time
file_list = sorted(os.listdir(directory_path), key=lambda x: get_file_modified_time(os.path.join(directory_path, x)), reverse=False)
#file_list.sort()  # not very neccesary

print(file_list)

for filename in file_list[start_index:start_index + num_files_to_process]:
    file_path = os.path.join(directory_path, filename)
    #print(file_path)
    if i >= start_index + num_files_to_process:
        break


    if os.path.isfile(file_path):
        sequence, angle, inputs, outputs, embedding = extract_input(file_path, model_embedd, tokenizer) # the key part that extracts all the information into a file
        if sequence is None or embedding is None or angle is None:
            print(f"Skipping {file_path} due to missing data")
            continue
        print(len(sequence))
        padded_sequence = pad_sequence(sequence)  # may not need this as im using a padding in the tokenizer itself
        sequences.append(padded_sequence)

        # Append the new data to the existing tensors
        embedding_length = min(embedding.size(1), 130)
        if i >= embeddings.size(0):
            embeddings = torch.cat([embeddings, torch.zeros(1, 130, 1024)], dim=0)
        embeddings[i, :embedding_length] = embedding[0, :, :]

        angle_length = min(angle.size(1), 130)
        if i >= angles.size(0):
            angles = torch.cat([angles, torch.zeros(1, 2, 130)], dim=0)
        angles[i, :, :angle_length] = angle[:, :]

        i += 1
        print(i)


# Save the updated tensors back to the files
torch.save(embeddings, embeddings_file)
torch.save(angles, angles_file)
with open(sequences_file, 'wb') as f: # you may not need to save the sequences
    pickle.dump(sequences, f)


# Path to the folder you want to download
#folder_path = '/content/map'
# Name of the output zip file
zip_name = 'inputs.zip' # downloads file with info

# Zip the folder
#shutil.make_archive(zip_name.replace('.zip', ''), 'zip', target_path)

# Download the zip file
#files.download(zip_name)

['1e0z.cif', '1ry4.cif', '1ug7.cif', '1whn.cif', '2aya.cif', '2d87.cif', '2db5.cif', '2dkp.cif', '2ebk.cif', '2ett.cif', '2fm4.cif', '2i4k.cif', '2jq5.cif', '2k0r.cif', '2k5t.cif', '2lr4.cif', '2ls0.cif', '2lvl.cif', '2mah.cif', '2ncn.cif', '2rlo.cif', '2rv8.cif', '2ytu.cif', '4ar0.cif', '5aiw.cif', '5h7u.cif', '5n9q.cif', '8e22.cif', '1cmo.cif', '1ddf.cif', '1eal.cif', '1pqn.cif', '1qkl.cif', '1uey.cif', '1w2q.cif', '1wh4.cif', '1wjr.cif', '1x5m.cif', '1zu1.cif', '2cso.cif', '2dmk.cif', '2ee7.cif', '2l1l.cif', '2loe.cif', '2mf7.cif', '2mpl.cif', '2nbo.cif', '2tmp.cif', '4uei.cif', '5m1u.cif', '7xrw.cif', '8oil.cif', '1rsf.cif', '1v5s.cif', '1x5e.cif', '1zo0.cif', '2a4h.cif', '2cr4.cif', '2dav.cif', '2if1.cif', '2jp2.cif', '2jvn.cif', '2k2v.cif', '2l3g.cif', '2l57.cif', '2lbc.cif', '2lln.cif', '2m26.cif', '3msp.cif', '7f7n.cif', '7knv.cif', '8sqx.cif', '1buq.cif', '1bw3.cif', '1fr0.cif', '1gio.cif', '1mh6.cif', '1psy.cif', '1uen.cif', '1wyj.cif', '2ai6.cif', '2dhj.cif', '2k4v.cif', '2k